In [5]:
# ! pip install beautifulsoup4
# ! pip install requests
# ! pip install pandas

Update:
+ Finish scrap the product listing page
+ Know where to find the SKU 
+ Need to scrap available switches and price of each product

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
# page_no = 1
# url_link = "https://mechanicalkeyboards.com/shop/index.php?pg="+str(page_no)+"&l=product_list&c=1&show=100"

# Shop listing page

In [7]:
def get_soup(url_link):
    return BeautifulSoup(requests.get(url_link).text, 'html.parser')


In [8]:
def get_number_of_page():
    soup = get_soup("https://mechanicalkeyboards.com/shop/index.php?pg=1&l=product_list&c=1&show=100")
    no_page = soup.select_one('div.blog-pagination span')
    return no_page.text.split()[-1]



In [9]:
PRODUCT_SOUP_TAG = "div.product-name a"
IMAGE_URL_TAG = 'div.imagebox li a img'

product_name_list = []
product_url_list = []
product_img_url_list = []

#Get the last page of website (APPLY FOR THIS URL ONLY)
#"https://mechanicalkeyboards.com/shop/index.php?pg=1&l=product_list&c=1&show=100"
no_page = int(get_number_of_page())

for i in range(1,no_page+1):
        url_link = "https://mechanicalkeyboards.com/shop/index.php?pg="+str(i)+"&l=product_list&c=1&show=100"
        product_list_soup = get_soup(url_link)
        product_soup = product_list_soup.select(PRODUCT_SOUP_TAG)
        image_link = product_list_soup.select(IMAGE_URL_TAG)
        for (name,url) in zip(product_soup, image_link):
                #Get element
                product_name = name.text
                product_url = name['href']
                full_image_url = "https://mechanicalkeyboards.com/shop/" + url['src']
                
                #Add to list
                product_name_list.append(product_name)
                product_url_list.append(product_url)
                product_img_url_list.append(full_image_url)



# Product Detail Page


In [98]:
def get_switch_table(soup):
    switch_price_dict = {
    "switch":{}
    }
    
    switch_table = soup.find('table', class_='edition_switch_list')
    switch_type = switch_table.find_all("td", class_='switch')
    switch_price= switch_table.find_all("td", class_='price')

    # switch_price['sku'] = sku.text
    for i in range(len(switch_type)):
        product_switch_sku = switch_type[i].find('div').text
        switch_type[i].find('div').decompose() 
        # print(f"switch_sku:{product_switch_sku}\nswitch_name:{switch_type[i].text}\nswitch_price:{switch_price[i].text}")
        switch_price_dict['switch'].update({"switch_sku":product_switch_sku, switch_type[i].text : switch_price[i].text })
    
    return switch_price_dict

def get_product_spec(soup):
    spec_table = soup.find('table', class_='v3_specs')
    spec_table_field = spec_table.find_all("td", class_='field')
    spec_table_value= spec_table.find_all("td", class_='value')
    
    product_spec = {
    "spec": {}
    }

    
    for i in range(len(spec_table)): 
        product_spec['spec'].update({spec_table_field[i].text : spec_table_value[i].text})
        
    return product_spec



In [99]:
product = {
    'name':"";
    
}


sample_url = "https://mechanicalkeyboards.com/shop/index.php?l=product_detail&p=5027"
img_url_base = "https://mechanicalkeyboards.com/shop/"
product_detail_soup = get_soup(sample_url)

product_name = product_detail_soup.find("h4", class_="name").text
product_sku = product_detail_soup.find('div',class_="product-id").find("span",class_='id').text
product_img_url = img_url_base+product_detail_soup.find('div',attrs={"style":'text-align: center;'}).find('a')['href']
product_switch_table = get_switch_table(product_detail_soup)
product_spec_table = get_product_spec(product_detail_soup)

print(product_spec_table)



{'spec': {'Brand': 'Ducky', 'Model': 'One 2 Mini', 'Size': '60%', 'Switch Stems': 'MX', 'Physical Layout': 'ANSI', 'Logical Layout': 'US QWERTY', 'Frame Color': 'White', 'Frame Top Material': 'Plastic', 'Frame Bottom Material': 'Plastic', 'Primary LED Color': 'RGB', 'Control LED Color': 'RGB', 'Hotswap Sockets': 'No', 'USB Key Rollover': 'Full', 'Multimedia Keys': 'Yes', 'Switch Mount Type': 'Plate', 'Built in Audio Port': 'No', 'Built in Mic Port': 'No', 'Interface(s)': 'USB-C', 'Windows Compatible': 'Yes', 'Mac Compatible': 'No', 'Linux Compatible': 'Yes', 'Dimensions': '4.25" x 11.50" x 1.50"', 'Weight': '2.00 lbs', 'Cord Length': '56 inches'}}


### FINAL STEP 

In [29]:
prod_list = []
for url in product_url_list[:2]:
    product_name = get_tag_detail_one(url, PRODUCT_NAME_TAG)
    sku = get_tag_detail_one(url, PRODUCT_SKU_TAG)

    switch = get_tag_detail(url, SWITCH_TAG )
    price = get_tag_detail(url, PRICE_TAG)
    
    #remove div tag in switch 
    for i in range(len(switch)):
        switch[i].find('div').decompose() 
        
    switch_list =[switch[i].text for i in range(len(switch))]
    switch_price_list = [price[i].text for i in range(len(price))]
      
    prod = create_product(product_name.text, sku.text, switch_list, switch_price_list)
    prod_list.append(prod)

#Export list of dict to json
with open("sample.json","w+") as f:
    json.dump(prod_list,f,indent=4)
    
    
    